In [ ]:
import time
import requests
import json
from web3 import Web3
from web3 import _utils # for error handling
from multiprocessing.pool import ThreadPool

import pandas as pd

# saving dictionaries and pd dataFrames
import pickle
def save_obj(obj, name):
    with open(name+'.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name):
    with open(name+'.pkl', 'rb') as f:
        return pickle.load(f)

In [2]:
# copy api key from https://dashboard.alchemy.com/
apiKey=""

# Use Web3 to process registration & renewal transactions

- I used web3 to get payments associated with registration and renewal events, which was used to estimate protocol revenue.

In [4]:
# connect to an ethereum node
alchemy_url = f"https://eth-mainnet.g.alchemy.com/v2/{apiKey}"
w3 = Web3(Web3.HTTPProvider(alchemy_url))
print(w3.isConnected())

True


In [3]:
# contracts, abis from etherscan
ens_nft_address = Web3.toChecksumAddress("0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85")
ens_nft_abi = '[{"inputs":[{"internalType":"contract ENS","name":"_ens","type":"address"},{"internalType":"bytes32","name":"_baseNode","type":"bytes32"}],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"approved","type":"address"},{"indexed":true,"internalType":"uint256","name":"tokenId","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"operator","type":"address"},{"indexed":false,"internalType":"bool","name":"approved","type":"bool"}],"name":"ApprovalForAll","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"controller","type":"address"}],"name":"ControllerAdded","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"controller","type":"address"}],"name":"ControllerRemoved","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"id","type":"uint256"},{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":false,"internalType":"uint256","name":"expires","type":"uint256"}],"name":"NameMigrated","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"id","type":"uint256"},{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":false,"internalType":"uint256","name":"expires","type":"uint256"}],"name":"NameRegistered","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"id","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"expires","type":"uint256"}],"name":"NameRenewed","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":true,"internalType":"uint256","name":"tokenId","type":"uint256"}],"name":"Transfer","type":"event"},{"constant":true,"inputs":[],"name":"GRACE_PERIOD","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"controller","type":"address"}],"name":"addController","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"tokenId","type":"uint256"}],"name":"approve","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"internalType":"uint256","name":"id","type":"uint256"}],"name":"available","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"owner","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"baseNode","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"controllers","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"ens","outputs":[{"internalType":"contract ENS","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"uint256","name":"tokenId","type":"uint256"}],"name":"getApproved","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"operator","type":"address"}],"name":"isApprovedForAll","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"isOwner","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"uint256","name":"id","type":"uint256"}],"name":"nameExpires","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"uint256","name":"tokenId","type":"uint256"}],"name":"ownerOf","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"uint256","name":"id","type":"uint256"},{"internalType":"address","name":"owner","type":"address"}],"name":"reclaim","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"uint256","name":"id","type":"uint256"},{"internalType":"address","name":"owner","type":"address"},{"internalType":"uint256","name":"duration","type":"uint256"}],"name":"register","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"uint256","name":"id","type":"uint256"},{"internalType":"address","name":"owner","type":"address"},{"internalType":"uint256","name":"duration","type":"uint256"}],"name":"registerOnly","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"controller","type":"address"}],"name":"removeController","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"uint256","name":"id","type":"uint256"},{"internalType":"uint256","name":"duration","type":"uint256"}],"name":"renew","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"renounceOwnership","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"tokenId","type":"uint256"}],"name":"safeTransferFrom","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"tokenId","type":"uint256"},{"internalType":"bytes","name":"_data","type":"bytes"}],"name":"safeTransferFrom","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"to","type":"address"},{"internalType":"bool","name":"approved","type":"bool"}],"name":"setApprovalForAll","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"resolver","type":"address"}],"name":"setResolver","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"internalType":"bytes4","name":"interfaceID","type":"bytes4"}],"name":"supportsInterface","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"tokenId","type":"uint256"}],"name":"transferFrom","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}]'

ens_token_address = Web3.toChecksumAddress("0xc18360217d8f7ab5e7c516566761ea12ce7f9d72")
ens_token_abi = '[{"inputs":[{"internalType":"uint256","name":"freeSupply","type":"uint256"},{"internalType":"uint256","name":"airdropSupply","type":"uint256"},{"internalType":"uint256","name":"_claimPeriodEnds","type":"uint256"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"claimant","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"Claim","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"delegator","type":"address"},{"indexed":true,"internalType":"address","name":"fromDelegate","type":"address"},{"indexed":true,"internalType":"address","name":"toDelegate","type":"address"}],"name":"DelegateChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"delegate","type":"address"},{"indexed":false,"internalType":"uint256","name":"previousBalance","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"newBalance","type":"uint256"}],"name":"DelegateVotesChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"bytes32","name":"merkleRoot","type":"bytes32"}],"name":"MerkleRootChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"inputs":[],"name":"DOMAIN_SEPARATOR","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"},{"internalType":"uint32","name":"pos","type":"uint32"}],"name":"checkpoints","outputs":[{"components":[{"internalType":"uint32","name":"fromBlock","type":"uint32"},{"internalType":"uint224","name":"votes","type":"uint224"}],"internalType":"struct ERC20Votes.Checkpoint","name":"","type":"tuple"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"claimPeriodEnds","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"address","name":"delegate","type":"address"},{"internalType":"bytes32[]","name":"merkleProof","type":"bytes32[]"}],"name":"claimTokens","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"subtractedValue","type":"uint256"}],"name":"decreaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"delegatee","type":"address"}],"name":"delegate","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"delegatee","type":"address"},{"internalType":"uint256","name":"nonce","type":"uint256"},{"internalType":"uint256","name":"expiry","type":"uint256"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"delegateBySig","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"delegates","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"blockNumber","type":"uint256"}],"name":"getPastTotalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"},{"internalType":"uint256","name":"blockNumber","type":"uint256"}],"name":"getPastVotes","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"getVotes","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"addedValue","type":"uint256"}],"name":"increaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"index","type":"uint256"}],"name":"isClaimed","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"merkleRoot","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"minimumMintInterval","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"dest","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"mint","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"mintCap","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"nextMint","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"}],"name":"nonces","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"numCheckpoints","outputs":[{"internalType":"uint32","name":"","type":"uint32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"permit","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"renounceOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_merkleRoot","type":"bytes32"}],"name":"setMerkleRoot","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"dest","type":"address"}],"name":"sweep","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"sender","type":"address"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"}]'

ens_dao_treasury_address =  Web3.toChecksumAddress("0xfe89cc7abb2c4183683ab71653c4cdc9b02d44b7")
ens_dao_treasury_abi = '[{"inputs":[{"internalType":"uint256","name":"minDelay","type":"uint256"},{"internalType":"address[]","name":"proposers","type":"address[]"},{"internalType":"address[]","name":"executors","type":"address[]"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"bytes32","name":"id","type":"bytes32"},{"indexed":true,"internalType":"uint256","name":"index","type":"uint256"},{"indexed":false,"internalType":"address","name":"target","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"},{"indexed":false,"internalType":"bytes","name":"data","type":"bytes"}],"name":"CallExecuted","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"bytes32","name":"id","type":"bytes32"},{"indexed":true,"internalType":"uint256","name":"index","type":"uint256"},{"indexed":false,"internalType":"address","name":"target","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"},{"indexed":false,"internalType":"bytes","name":"data","type":"bytes"},{"indexed":false,"internalType":"bytes32","name":"predecessor","type":"bytes32"},{"indexed":false,"internalType":"uint256","name":"delay","type":"uint256"}],"name":"CallScheduled","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"bytes32","name":"id","type":"bytes32"}],"name":"Cancelled","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"oldDuration","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"newDuration","type":"uint256"}],"name":"MinDelayChange","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"bytes32","name":"role","type":"bytes32"},{"indexed":true,"internalType":"bytes32","name":"previousAdminRole","type":"bytes32"},{"indexed":true,"internalType":"bytes32","name":"newAdminRole","type":"bytes32"}],"name":"RoleAdminChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"bytes32","name":"role","type":"bytes32"},{"indexed":true,"internalType":"address","name":"account","type":"address"},{"indexed":true,"internalType":"address","name":"sender","type":"address"}],"name":"RoleGranted","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"bytes32","name":"role","type":"bytes32"},{"indexed":true,"internalType":"address","name":"account","type":"address"},{"indexed":true,"internalType":"address","name":"sender","type":"address"}],"name":"RoleRevoked","type":"event"},{"inputs":[],"name":"DEFAULT_ADMIN_ROLE","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"EXECUTOR_ROLE","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"PROPOSER_ROLE","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"TIMELOCK_ADMIN_ROLE","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"id","type":"bytes32"}],"name":"cancel","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"target","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"bytes","name":"data","type":"bytes"},{"internalType":"bytes32","name":"predecessor","type":"bytes32"},{"internalType":"bytes32","name":"salt","type":"bytes32"}],"name":"execute","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address[]","name":"targets","type":"address[]"},{"internalType":"uint256[]","name":"values","type":"uint256[]"},{"internalType":"bytes[]","name":"datas","type":"bytes[]"},{"internalType":"bytes32","name":"predecessor","type":"bytes32"},{"internalType":"bytes32","name":"salt","type":"bytes32"}],"name":"executeBatch","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getMinDelay","outputs":[{"internalType":"uint256","name":"duration","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"role","type":"bytes32"}],"name":"getRoleAdmin","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"id","type":"bytes32"}],"name":"getTimestamp","outputs":[{"internalType":"uint256","name":"timestamp","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"role","type":"bytes32"},{"internalType":"address","name":"account","type":"address"}],"name":"grantRole","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"role","type":"bytes32"},{"internalType":"address","name":"account","type":"address"}],"name":"hasRole","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"target","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"bytes","name":"data","type":"bytes"},{"internalType":"bytes32","name":"predecessor","type":"bytes32"},{"internalType":"bytes32","name":"salt","type":"bytes32"}],"name":"hashOperation","outputs":[{"internalType":"bytes32","name":"hash","type":"bytes32"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"address[]","name":"targets","type":"address[]"},{"internalType":"uint256[]","name":"values","type":"uint256[]"},{"internalType":"bytes[]","name":"datas","type":"bytes[]"},{"internalType":"bytes32","name":"predecessor","type":"bytes32"},{"internalType":"bytes32","name":"salt","type":"bytes32"}],"name":"hashOperationBatch","outputs":[{"internalType":"bytes32","name":"hash","type":"bytes32"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"bytes32","name":"id","type":"bytes32"}],"name":"isOperation","outputs":[{"internalType":"bool","name":"pending","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"id","type":"bytes32"}],"name":"isOperationDone","outputs":[{"internalType":"bool","name":"done","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"id","type":"bytes32"}],"name":"isOperationPending","outputs":[{"internalType":"bool","name":"pending","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"id","type":"bytes32"}],"name":"isOperationReady","outputs":[{"internalType":"bool","name":"ready","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"role","type":"bytes32"},{"internalType":"address","name":"account","type":"address"}],"name":"renounceRole","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"role","type":"bytes32"},{"internalType":"address","name":"account","type":"address"}],"name":"revokeRole","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"target","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"bytes","name":"data","type":"bytes"},{"internalType":"bytes32","name":"predecessor","type":"bytes32"},{"internalType":"bytes32","name":"salt","type":"bytes32"},{"internalType":"uint256","name":"delay","type":"uint256"}],"name":"schedule","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"targets","type":"address[]"},{"internalType":"uint256[]","name":"values","type":"uint256[]"},{"internalType":"bytes[]","name":"datas","type":"bytes[]"},{"internalType":"bytes32","name":"predecessor","type":"bytes32"},{"internalType":"bytes32","name":"salt","type":"bytes32"},{"internalType":"uint256","name":"delay","type":"uint256"}],"name":"scheduleBatch","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes4","name":"interfaceId","type":"bytes4"}],"name":"supportsInterface","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"newDelay","type":"uint256"}],"name":"updateDelay","outputs":[],"stateMutability":"nonpayable","type":"function"},{"stateMutability":"payable","type":"receive"}]'

ens_registrar_address = Web3.toChecksumAddress("0x283af0b28c62c092c9727f1ee09c02ca627eb7f5")
ens_registrar_abi = '[{"inputs":[{"internalType":"contract BaseRegistrar","name":"_base","type":"address"},{"internalType":"contract PriceOracle","name":"_prices","type":"address"},{"internalType":"uint256","name":"_minCommitmentAge","type":"uint256"},{"internalType":"uint256","name":"_maxCommitmentAge","type":"uint256"}],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"name","type":"string"},{"indexed":true,"internalType":"bytes32","name":"label","type":"bytes32"},{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":false,"internalType":"uint256","name":"cost","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"expires","type":"uint256"}],"name":"NameRegistered","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"string","name":"name","type":"string"},{"indexed":true,"internalType":"bytes32","name":"label","type":"bytes32"},{"indexed":false,"internalType":"uint256","name":"cost","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"expires","type":"uint256"}],"name":"NameRenewed","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"oracle","type":"address"}],"name":"NewPriceOracle","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"constant":true,"inputs":[],"name":"MIN_REGISTRATION_DURATION","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"string","name":"name","type":"string"}],"name":"available","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"bytes32","name":"commitment","type":"bytes32"}],"name":"commit","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"name":"commitments","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"isOwner","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"string","name":"name","type":"string"},{"internalType":"address","name":"owner","type":"address"},{"internalType":"bytes32","name":"secret","type":"bytes32"}],"name":"makeCommitment","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":true,"inputs":[{"internalType":"string","name":"name","type":"string"},{"internalType":"address","name":"owner","type":"address"},{"internalType":"bytes32","name":"secret","type":"bytes32"},{"internalType":"address","name":"resolver","type":"address"},{"internalType":"address","name":"addr","type":"address"}],"name":"makeCommitmentWithConfig","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":true,"inputs":[],"name":"maxCommitmentAge","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"minCommitmentAge","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"string","name":"name","type":"string"},{"internalType":"address","name":"owner","type":"address"},{"internalType":"uint256","name":"duration","type":"uint256"},{"internalType":"bytes32","name":"secret","type":"bytes32"}],"name":"register","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"internalType":"string","name":"name","type":"string"},{"internalType":"address","name":"owner","type":"address"},{"internalType":"uint256","name":"duration","type":"uint256"},{"internalType":"bytes32","name":"secret","type":"bytes32"},{"internalType":"address","name":"resolver","type":"address"},{"internalType":"address","name":"addr","type":"address"}],"name":"registerWithConfig","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"internalType":"string","name":"name","type":"string"},{"internalType":"uint256","name":"duration","type":"uint256"}],"name":"renew","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[],"name":"renounceOwnership","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"internalType":"string","name":"name","type":"string"},{"internalType":"uint256","name":"duration","type":"uint256"}],"name":"rentPrice","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"uint256","name":"_minCommitmentAge","type":"uint256"},{"internalType":"uint256","name":"_maxCommitmentAge","type":"uint256"}],"name":"setCommitmentAges","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"contract PriceOracle","name":"_prices","type":"address"}],"name":"setPriceOracle","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"internalType":"bytes4","name":"interfaceID","type":"bytes4"}],"name":"supportsInterface","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"internalType":"string","name":"name","type":"string"}],"name":"valid","outputs":[{"internalType":"bool","name":"","type":"bool"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":false,"inputs":[],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}]'


In [5]:
# initializing representations of contracts
ens_token_contract = w3.eth.contract(address = ens_token_address, abi = ens_token_abi)
ens_nft_contract = w3.eth.contract(address = ens_nft_address, abi = ens_nft_abi)
ens_dao_treasury_wallet = w3.eth.contract(address = ens_dao_treasury_address, abi = ens_dao_treasury_abi)
ens_registrar_contract = w3.eth.contract(address = ens_registrar_address, abi = ens_registrar_abi)

#### Registration Events

In [8]:
# # get all registration logs
# end_block = w3.eth.block_number
# start_block = end_block - 6000 * 365 # get one year assuming roughly 6k blocks/day

# # dict keyed by block to make filling in chunks easier
# logs_all=list()
# block = start_block

In [ ]:
# # get all registration events
# while block <= end_block:
#     logs_i = ens_nft_contract.events.NameRegistered().getLogs(fromBlock = block - 2000, toBlock = min(block,end_block+2001))
#     logs_all+=list(logs_i)
#     save_obj(logs_all, 'registration_logs')
#     block+=2001
#     if block%100==0:
#         print(f'{block}, {block-start_block}/{end_block-start_block} ({(block-start_block)/(end_block-start_block)}% done)')

In [ ]:
# # load the logs from loop above
# registration_logs=load_obj('registration_logs')

In [ ]:
# # get and decode all registration events (moved to a funuction that can be run for chunks in parallel)
# for log in registration_logs:
#     i+=1
#     tx_hash = log.transactionHash
#     if not tx_hash.hex() in d.keys():
#         tx_receipt = w3.eth.get_transaction_receipt(tx_hash)
#         register_event = ens_registrar_contract.events.NameRegistered().processReceipt(tx_receipt,errors=_utils.events.EventLogErrorFlags.Discard)

#         d[register_event[0]['transactionHash'].hex()] = {
#         'blockNumber':register_event[0]['blockNumber'],
#         'owner': register_event[0]['args']['owner'], # not included in renew events
#         'name': register_event[0]['args']['name'],
#         'cost': register_event[0]['args']['cost'],
#         'expires': register_event[0]['args']['expires']
#         }
        
#         save_obj(d, 'registration_logs_dict')

#     if i%1000==0: print(f'{i},{len(d)} / {len(registration_logs)} (around {i/len(registration_logs)*100}% done)')


In [ ]:
# # function to do this in parallel
# def get_registration_dicts(chunk):
#     t1=time.time()
#     d=dict()
#     first_hash = chunk[0].transactionHash.hex()[0:7]
#     i=0
#     for log in chunk:
#         i+=1
#         tx_hash = log.transactionHash
#         if not tx_hash.hex() in d.keys():
#             tx_receipt = w3.eth.get_transaction_receipt(tx_hash)
#             register_event = ens_registrar_contract.events.NameRegistered().processReceipt(tx_receipt,errors=_utils.events.EventLogErrorFlags.Discard)

#             d[register_event[0]['transactionHash'].hex()] = {
#             'blockNumber':register_event[0]['blockNumber'],
#             'owner': register_event[0]['args']['owner'], # not included in renew events
#             'name': register_event[0]['args']['name'],
#             'cost': register_event[0]['args']['cost'],
#             'expires': register_event[0]['args']['expires']
#             }
#         if i%200==0: print(f'{first_hash}, {i},{len(d)} / {len(chunk)} (around {i/len(chunk)*100}% done)')
#     path = f'registration_logs_dict_{tx_hash.hex()}'
#     save_obj(d, path)
#     print(f'{first_hash} done in {time.time()-t1}')
#     return path

In [ ]:
# # TODO: run & monitor from terminal instances within jupyter
# # get index ranges

# # left off around 67000
# ranges=[i for i in range(67000,int(len(registration_logs)),int(round(len(registration_logs)/1000)))]
# ranges.append(len(registration_logs))

# # get list of events to pass in each chunk
# chunks = [registration_logs[ranges[i]:ranges[i+1]] for i in range(len(ranges)-1)]

In [ ]:
# pool = ThreadPool()

In [ ]:
# results = pool.map(get_registration_dicts, chunks)

In [ ]:
# # 970 chunks
# # 350s, 6min/ chunk
# # 8 parallel chunks
# # 970 * 6 / 60 / 8 = around 10h
# save_obj(results,'registration_logs_dics_names')

In [ ]:
# # TODO: Process dict into df
# # load dist from serial run
# load_obj('registration_logs_dict') #dict()
# # load each of registration_logs_dics_names
# ... 
# for all chunks..
# d_chunk = load the chunk,
# d = d.update(d_chunk)
# remove chunk from memory

#### Renewal Events

In [ ]:
# # # get all NameRenewed logs

# # params for getting registration and renewal events
# # (run before either of the following blocks)
# end_block = w3.eth.block_number
# start_block = end_block - 6000 * 365 # get one year assuming roughly 6k blocks/day

# # dict keyed by block to make filling in chunks easier
# logs_all=list()
# block = start_block

# while block <= end_block:
#     logs_i = ens_nft_contract.events.NameRenewed().getLogs(fromBlock = block - 2000, toBlock = min(block,end_block+2001))
#     logs_all+=list(logs_i)
#     save_obj(logs_all, 'renewal_logs')
#     block+=2001
#     if block%100==0:
#         print(f'{block}, {block-start_block}/{end_block-start_block} ({(block-start_block)/(end_block-start_block)}% done)')

In [ ]:
# process renewal logs, save processed logs
# load logs from loop above
# renewal_logs=load_obj('renewal_logs')

In [ ]:
# # convert them to a dict

# d=dict()
# for log in renewal_logs:
#     tx_hash = log.transactionHash
#     tx_receipt = w3.eth.get_transaction_receipt(tx_hash)
#     register_event = ens_registrar_contract.events.NameRenewed().processReceipt(tx_receipt)
    
#     d[register_event[0]['transactionHash'].hex()] = {
#     'blockNumber':register_event[0]['blockNumber'],
#     # 'owner': register_event[0]['args']['owner'], # not included in renew events
#     'name': register_event[0]['args']['name'],
#     'cost': register_event[0]['args']['cost'],
#     'expires': register_event[0]['args']['expires']
#     }

# save_obj(d, 'renewal_logs_dict')

In [ ]:
# load the dict created abve
# tmp = load_obj('renewal_logs_dict')

In [ ]:
len(renewal_logs),len(tmp)

In [ ]:
# TODO: convert to df, and analyze!

## Registration & Renwal Analyses

In [ ]:
# TODO;
# - process registration and renewal logs into dicts
# - process them into dfs
# - analyses!
# -- payments for each
# -- registration durations over time (might need nft metadata from below for this)
# -- other protocol revenue metrics in prompt...

# Use Alchemy for NFT transaction and holder data

In [13]:
# TODO: 
# check each alchamey function...

# - change token balances to=true? and see what is returned?

# - write function for getNFTsForCollection
# - process output of getNFTsForCollection


# revenue and income projections.. 
# revenue data from alchamey & knoeledge of registration costs;
# name_length, registration date, & expiration date can be used to estimate future revenue?
# assuming everyone re-registers at the same prices for the same duration ; 
# revenue = annual price|name_langth * (expiration date - registration date) in years

# estimate
# - new users / new registrations & project this with a regression model?
# - historical re-registration rate
# - quarterly profits from registrations, & from re-registrations?


In [14]:
# Alchamey requests api;

base_url = f"https://eth-mainnet.g.alchemy.com/nft/v2/{apiKey}"

# collection -> owners & owned tokens
def getOwnersForCollection():
    # docs: https://docs.alchemy.com/reference/getownersforcollection
    # first page
    first_page_url = base_url+ f"/getOwnersForCollection?contractAddress={ens_nft_address}&withTokenBalances=true"
    headers = {"accept": "application/json"}
    response = requests.get(first_page_url, headers=headers) 

    # response.json()['ownerAddresses'] is a list with len() usually around 50k
    owner_addresses_page = response.json()['ownerAddresses']
    owner_addresses_all = owner_addresses_page

    # subsequent pages (todo:add more conditions avoid infinite loops)
    while True:
        page_key_old = response.json()['pageKey']
        next_page_url = first_page_url + f"&pageKey={page_key_old}"
        response = requests.get(next_page_url, headers=headers) 
        owner_addresses_page = response.json()['ownerAddresses']
        owner_addresses_all+=owner_addresses_page
        print(len(owner_addresses_all),page_key_old) # feedback to monitor
        if 'pageKey' not in response.json():
            print('no more page keys')
            return owner_addresses_all

# owner -> NFTs owned
# https://docs.alchemy.com/reference/getnfts
def getNFTs(owner):
    first_page_url = base_url + f"/getNFTs?owner={owner}&contractAddresses[]={ens_nft_address}&withMetadata=true"
    headers = {"accept": "application/json"}
    response = requests.get(first_page_url, headers=headers)
    response_json = response.json()

    owner_nfts_all = response_json['ownedNfts']
    while len(owner_nfts_all) < response_json['totalCount']:
            page_key = response_json['pageKey']
            next_page_url = first_page_url + f"&pageKey={page_key}"
            response = requests.get(next_page_url, headers=headers) # introduces dupes bc of page size being 100, need to dedupe later
            response_json=response.json()
            owner_nfts_all += response_json['ownedNfts']
            print(f"{len(owner_nfts_all)} / {response_json['totalCount']}",page_key) # feedback to monitor

    return owner_nfts_all # may have dupes -- could dedupe by processing into a dict keyed by tokenId

def process_nft_list(nft_list):
    nfts_df = pd.DataFrame()
    for nft in nft_list:
        if 'metadata' in nft.keys():
            if 'attributes' in nft['metadata'].keys():
                if nft['metadata']['attributes']:
                    tmp_df=pd.DataFrame(nft['metadata']['attributes']).set_index('trait_type')
                    tmp_df=tmp_df.T.loc[['value']] # transpose for easier formatting
                else: tmp_df=pd.DataFrame()
            else: tmp_df=pd.DataFrame()
        else: tmp_df=pd.DataFrame()

        if 'title' in nft.keys(): 
            tmp_df['title'] = nft['title']
        else: tmp_df['title'] = None

        if 'id' in nft.keys():
            if 'tokenId' in nft['id'].keys():
                tmp_df['id'] = nft['id']['tokenId']
        else: tmp_df['id'] = None

        # if 'owner' in nft.keys():
        #     tmp_df['owner']=owner
        # else: tmp_df['owner']=owner

        nfts_df=nfts_df.append(tmp_df)

    nfts_df=nfts_df.drop_duplicates()
    return nfts_df

# process the list of nfts with metadata and dupes

# collection -> NFT info
# https://docs.alchemy.com/reference/getnftsforcollection
# skipped bc relevant info was returned by getNFTs
# # getNFTsForCollection
# url = f"https://eth-mainnet.g.alchemy.com/nft/v2/{apiKey}/getNFTsForCollection?contractAddress={ens_nft_address}&withMetadata=true"
# headers = {"accept": "application/json"}
# response = requests.get(url, headers=headers)

# NFT sales
# https://docs.alchemy.com/reference/getnftsales


## Owners and domains owned

## Method 1: owners -> NFTs -> metadata

### Get owners

In [15]:
# get owners
owner_addresses_all = getOwnersForCollection()

19926 MHgwM2QzMmY4OGZjOGIxMGFmMGNiYmJkZjNjOWE3NGEzNmMzNjhlZWZjOjB4N2U5NzMzNjQ1MzZhMmMwNjRjOTFiNjlkMzI3NWZjYWE3NzY4MzUwYTI0NmRhZGJiZTZmOGQyNTU4NzZhMTgxNDpmYWxzZQ==
29783 MHgwOGNmOTk2MGM5Mjc3N2JkNmE1OTZhNGYzZGMzNDIxMGQxNjZjODdmOjB4NjAyYzA3NjgyZDg2Y2RkMmE2OTkwM2VjYWUxZjgzNWE0M2VmMjU1MmIwMTk2YWE2YWU0OTA2MGQ1NThlOGViYTpmYWxzZQ==
41968 MHgwZDU1ZWE4OGIwYjc2OWY5Mzg3M2Y1Yjk3ODVhYjgwNGQzNzZiZjA4OjB4YTMxMjVkMzY5MDZmZDdiYzhjYjBlNDBkYzM3YjgyOTlhZDhiYTBhMzc4ODliYTI1ZDFjNjJkZTg5M2JlOTJmMDpmYWxzZQ==
53778 MHgxMmYzZDU5MzJkMmZkZmJjNGQ2ODU3ZGE5OGIxNDNmNTRmZjVmYjMzOjB4OWI0OGUwYWNiNzg4YzhlNjQ5YTc4MWZhYmMxZmIxYjhlYjMzZGEzYzM0MzlmZjM0Y2ZiMjU2NzZiMjUxMjM4YzpmYWxzZQ==
64844 MHgxODU1MWJlMmRiZGUwM2NhYjc1YzU4NWVkYmVkZDgwZmUxZTVlMWYzOjB4NjkwMmNmMDc4YWY4ZTExNWFjYjcwNjNlYWQ5NWEyODA3MzAxYTY2NzA1ZmRiNmFlMGJhOGJjZjA0ZmUxZTgyODpmYWxzZQ==
76405 MHgxZDZmMmYwMzU2YjNkZWZhZGYxNGIxYTBmOGEzZGNkYTg5MzY3ZDY4OjB4Y2E4Nzc1YTJkYzhlNWFhZjBmNzZhYmZkZTRlYTgwMzNlOWM5Y2VhMDQ4ZTliMmI3YjJjYjU1MjljMzMxMTgxYjpmYWxzZQ==
86700 MHgyMmRmMDA2N2Ew

In [16]:
# efficiently process info into a long data frame;
# step 1) convert to a dict with key:owner id, value: list of owned nfts
all_dict = dict()
i=-1
for owner in owner_addresses_all:
    i+=1
    if i%100000==0: print(f'{i}/{len(owner_addresses_all)}')
    all_dict[owner['ownerAddress']] = [[i['tokenId'] for i in owner['tokenBalances']]]
print('done.')

0/554770
100000/554770
200000/554770
300000/554770
400000/554770
500000/554770
done.


In [17]:
len(all_dict.keys()) # verify number of owner addresses

554649

In [18]:
# step 2) convert dict of lists into a df of lists
tmp=pd.DataFrame.from_dict(all_dict,orient='index',columns=['nft_ids'])
# step 3) explode the lists into rows for each (owner,nft id) pair, rename index coluumn
tmp=tmp.explode('nft_ids').reset_index().rename(columns={'index':'address_owner'})
# save for later usage
tmp.to_pickle(path = '/Users/macstrelioff/Documents/git/crypto_public/ENS NFT analyses/owners_and_owned_nft_ids_df')

In [19]:
# load the saved result instead of re-running code above
owners_and_owned_nft_ids_df = pd.read_pickle('owners_and_owned_nft_ids_df')

In [22]:
owners_and_owned_nft_ids_df.head()

,address_owner,nft_ids
0,0x0000000000000000000000000000000000000001,0x048fcb12e6268ffe230ec5cd4148909d78c83d66dd64...
1,0x0000000000000000000000000000000000000001,0x056ca2314f8a6e8efe226d76ed83647670d0f2cb0ed8...
2,0x0000000000000000000000000000000000000001,0x06f16259602eb8442dbf3aa9e163eec8557f643ec435...
3,0x0000000000000000000000000000000000000001,0x0d73a424a96fa74a63677d10aecefbe1d1573495ce66...
4,0x0000000000000000000000000000000000000001,0x0d9b90614509308bb0ce53096c8410e953af7ca492a1...


In [21]:
# some check number of nfts and owners
owners_and_owned_nft_ids_df.shape,owners_and_owned_nft_ids_df.address_owner.nunique()

((2546310, 2), 554649)

### Get nfts by collection id (to be matched to owners later)

In [48]:
# # for getting metadata on all nfts in the collection

# first_page_url = base_url + f"/getNFTsForCollection?contractAddress={ens_nft_address}&withMetadata=true"
# headers = {"accept": "application/json"}
# response = requests.get(first_page_url, headers=headers)
# nfts_page = response.json()['nfts']

# nfts_all = nfts_page

# # for pagniation 
# i=-1

In [62]:
startToken = nfts_all[-1]['id']['tokenId']

In [71]:
len(nfts_all)

1007000

In [ ]:
while True:
    i+=1
    if len(nfts_all)%1000==0 or i%10==0: 
        save_obj(nfts_all, 'nfts_all')
        print(f'{i}, {len(nfts_all)}, {len(nfts_page)}, aprox. {len(nfts_all)/2546310*100}% done, {startToken}') # feedback to monitor
    startToken = nfts_all[-1]['id']['tokenId'] # response.json()['nextToken']
    next_page_url = first_page_url + f"&startToken={startToken}"
    response = requests.get(next_page_url, headers=headers) 
    if response.status_code != 200: continue # try again if status cod isnt 200
    nfts_page = response.json()['nfts']
    nfts_all+=nfts_page
    if 'nextToken' not in response.json():
        print('no more page keys')
        # return nfts_all

10078, 1007000, 100, aprox. 39.547423526593384% done, 0x622ddf708142af06c69cd6afc762b1f37720ed72a6ebf865f078f96383f8364f
10080, 1007200, 100, aprox. 39.55527802977642% done, 0x62330b10e78172859d47e2c258bd47781a42b602e6e132a9704259ec1be80daf
10088, 1008000, 100, aprox. 39.58669604250857% done, 0x6246e3a8d30f8eed17b541804cc7327fd2088a9720f01e5036a0740362e481d7
10090, 1008200, 100, aprox. 39.59455054569161% done, 0x624c02136d5039f5b9512239eed200de844071152977ef4069eff725d227f7bb
10098, 1009000, 100, aprox. 39.62596855842376% done, 0x626113a6fbed8617d25055ecb90ec99c949bf4834dcf1c865b1b26b9fd571a29
10100, 1009200, 100, aprox. 39.63382306160679% done, 0x6265e740921f02e3242c73abd48d110f13861e345ed9369e4e3e5dff6718c641
10108, 1010000, 100, aprox. 39.665241074338944% done, 0x627adcf4ede4fdb095ace1ecfd99d81b3e1d4ec68419f88e6279835ec6844eef
10110, 1010200, 100, aprox. 39.673095577521984% done, 0x628068e72baf5569b3e2f4df51bf6c7383e4daa7a8956fd1a210d5b03f4d4faa
10118, 1011000, 100, aprox. 39.704513

### join owners and nft ids to nft metadata kyed by nft id

## Method 2: Directly get owners and NFTs owned

In [ ]:
# get all owner addresses
owner_addresses_all = getOwnersForCollection()

In [ ]:
# check number of owners
len(owner_addresses_all) # = 554110, vs 554080 on etherscan https://etherscan.io/token/0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85#balances

In [ ]:
# get all nfts owned by each owner & format into a pandas df
# df = pd.DataFrame()
df = pd.read_pickle('owners_and_their_nfts') # to start where this left off...
# i = 0
i=12000 # last printed id

In [ ]:
# TODO: 
# - import pickle, 
# - save df so it can be accessed quickly later

# - for faster loop, save each wallet's df, then later concatinate them all.

while i <= len(owner_addresses_all): # looping over indecies makes it easier to continue where i left off if there is an error
    owner_address = owner_addresses_all[i]
    if i%500 == 0: 
        df.to_pickle(path = '/Users/macstrelioff/Documents/git/crypto_public/ENS NFT analyses/owners_and_their_nfts')
        print(f'{i} / {len(owner_addresses_all)}')
    if owner_address == '0x000000000000000000000000000000000000dead': continue # skip dead address
    try:
        nft_list = getNFTs(owner_address)
        nft_df = process_nft_list(nft_list) # dead address gave error
    except: 
        nft_df = pd.DataFrame()
    nft_df['owner']=owner_address
    df=df.append(nft_df)
    i+=1
    df=df.drop_duplicates()


# get nft sales

In [ ]:
# # start about y ear ago, assuming about 6k blocks/day
# startBlock = w3.eth.block_number - 6000 * 365 * 4 # should get all trades since ens was created may 2019 on opensea
# print(f'startBlock = {startBlock}')
# startLogIndex=0

# latestBlock = w3.eth.block_number
# print(f'latestBlock = {latestBlock}')

# url = base_url + f"/getNFTSales?startBlock={startBlock}&startLogIndex={startLogIndex}&startBundleIndex=0&ascendingOrder=true&marketplace=seaport&contractAddress={ens_nft_address}&buyerIsMaker=true"
# headers = {"accept": "application/json"}
# response = requests.get(url, headers=headers)

# returned_sales_df = pd.DataFrame(response.json()['nftSales'])
# nft_sales_df = returned_sales_df

# newBlockNumber = response.json()['nftSales'][-1]['blockNumber']
# newLogIndex = response.json()['nftSales'][-1]['logIndex']

# lastPrintBlock = newBlockNumber

# while newBlockNumber < latestBlock and len(returned_sales_df)>1:
    
#     if newBlockNumber - lastPrintBlock > 6000*7: # print feedback for about each week worth of blocks
#         lastPrintBlock = newBlockNumber
#         print(f'block {newBlockNumber} / {latestBlock} ({(newBlockNumber-startBlock) / (latestBlock-startBlock) * 100}% done)')
    
#     url = base_url + f"/getNFTSales?startBlock={newBlockNumber}&startLogIndex={newLogIndex}&startBundleIndex=0&ascendingOrder=true&marketplace=seaport&contractAddress={ens_nft_address}&buyerIsMaker=true"
#     headers = {"accept": "application/json"}
#     response = requests.get(url, headers=headers)
#     returned_sales_df = pd.DataFrame(response.json()['nftSales'])
#     nft_sales_df = nft_sales_df.append(returned_sales_df)
    
#     newBlockNumber = response.json()['nftSales'][-1]['blockNumber']
#     newLogIndex = response.json()['nftSales'][-1]['logIndex']
# print('done.')


In [ ]:
# # NOTE possibly missing earlier data
# # startBlock = 6880054 # 4 years ago
# # latestBlock = 15640054
# # block 15009916 / 15640054 (92.80664383561644% done) # first finishd chunk of blocks, seems to skip ~90% of early blocks

# save result of loop
# nft_sales_df.to_pickle(path = '/Users/macstrelioff/Documents/git/crypto_public/ENS NFT analyses/nft_sales_df')

In [ ]:
# load the saved result instead of re-running code above
nft_sales_df = pd.read_pickle('nft_sales_df')